In [1]:
import os 
import time
import copy
from tqdm import tqdm

import numpy as np 
import matplotlib.pyplot as plt
import scipy
import pandas as pd

from joblib import Parallel, delayed
from numba import jit, njit, prange

np.random.seed(0)

# Define macros

# Physical constants
K=7 # number of fermionic modes
J=4 # ~"energy scale"
Q=4 # order of coupling
N = 2*K # number of fermions
N_DIM = 2**K # Hilbert space dimensions

N_SAMPLES = 100 # number of samples to generate

## 1. Define fermionic modes

In [2]:
cr = np.array([[0,1],[0,0]])
an = np.array([[0,0],[1,0]])
id = np.identity(2)
id2 = np.array([[-1,0],[0,1]])

def c(n):
    factors = [id for i in range(n-1)]+[cr]+[id2 for i in range(K-n)]
    out = factors[0]
    for i in range(1, K):
        out = np.kron(out,factors[i])
    return out

def cd(n):
    factors = [id for i in range(n-1)]+[an]+[id2 for i in range(K-n)]
    out = factors[0]
    for i in range(1, K):
        out = np.kron(out,factors[i])
    return out

### 1.a. Check that fermionic modes satisfy algebra

$\{c_i, c_j\}=\{c_i^\dagger, c_j^\dagger\}=0$,     $\{c_i,c_j^\dagger\}=\delta_{ij}$

In [3]:
def anticommutator(a,b):
    return a@b+b@a

c_algebra_satisfied = True
for i in range(1,K+1):
    c_i = c(i)
    cd_i = cd(i)

    for j in range(1,K+1):
        c_j = c(j)
        cd_j = cd(j)

        ac_ci_cj = anticommutator(c_i, c_j)
        if not np.allclose(ac_ci_cj, np.zeros((N_DIM, N_DIM))):
            c_algebra_satisfied = False

        ac_cdi_cdj = anticommutator(cd_i, cd_j)
        if not np.allclose(ac_cdi_cdj, np.zeros((N_DIM, N_DIM))):
            c_algebra_satisfied = False

        ac_ci_cdj = anticommutator(c_i, cd_j)
        if i==j:
            if not np.allclose(ac_ci_cdj, np.identity(N_DIM)):
                c_algebra_satisfied = False
        else:
            if not np.allclose(ac_ci_cdj, np.zeros((N_DIM, N_DIM))):
                c_algebra_satisfied = False

        
print(f"Fermionic mode algebra satisfied: {c_algebra_satisfied}")

Fermionic mode algebra satisfied: True


## 2. Define fermions

In [4]:
# Compute first N psi's
psi = np.zeros((N, N_DIM, N_DIM), dtype=np.complex128)
for i in range(1,K+1):
    psi[2*(i-1)] = (c(i)+cd(i))/np.sqrt(2)
    psi[2*(i-1)+1] = (c(i)-cd(i))*(-1j/np.sqrt(2))

### 2.a. Check that fermions satisfy algebra

$\{\psi_i, \psi_j\} = \delta_{ij}$

In [5]:
psi_algebra_satisfied = True
for i in range(N):
    psi_i = psi[i]

    for j in range(N):
        psi_j = psi[j]

        ac_pi_pj = anticommutator(psi_i, psi_j)

        if i==j:
            if not np.allclose(ac_pi_pj, np.identity(N_DIM)):
                psi_algebra_satisfied = False
                
        else:
            if not np.allclose(ac_pi_pj, np.zeros((N_DIM, N_DIM))):
                psi_algebra_satisfied = False
        
print(f"Fermion algebra satisfied: {psi_algebra_satisfied}")

Fermion algebra satisfied: True


## 3. Define different Hamiltonian-creation functions

### 3. a. Pre-compute pairwise inner-products of fermions

Saves us some time and compute

In [6]:
# Compute pairwise inner products
psi_pairs = np.zeros((N, N, N_DIM, N_DIM), dtype=np.complex128)
for i in range(N):
    for j in range(i+1, N):
        psi_pairs[i, j] = psi[i]@psi[j]

### 3.b Define different Hamiltonian-creation functions to be tested

In [7]:
def H4_base(js): #js being the random coefficients

    # Compute Hamiltonian
    H = np.zeros((N_DIM, N_DIM), dtype=np.complex128)
    for i in range(N):
        for j in range(i+1, N):
            for k in range(j+1, N):
                for l in range(k+1, N):
                    H += (1j**(Q/2))*js[i, j, k, l]*psi_pairs[i,j].dot(psi_pairs[k,l])

    return H

@jit(nopython=True)
def H4_njit(js): #js being the random coefficients

    # Compute Hamiltonian|
    H = np.zeros((N_DIM, N_DIM), dtype=np.complex128)
    for i in range(N):
        for j in range(i+1, N):
            for k in range(j+1, N):
                for l in range(k+1, N):
                    H += (1j**(Q/2))*js[i, j, k, l]*psi_pairs[i,j].dot(psi_pairs[k,l])

    return H

@jit(nopython=True, parallel=True)
def H4_njit_parallel(js): #js being the random coefficients

    # Compute Hamiltonian
    H = np.zeros((N_DIM, N_DIM), dtype=np.complex128)
    for i in range(N):
        for j in range(i+1, N):
            for k in range(j+1, N):
                for l in range(k+1, N):
                    H += (1j**(Q/2))*js[i, j, k, l]*psi_pairs[i,j].dot(psi_pairs[k,l])

    return H

@jit(nopython=True, parallel=True)
def H4_njit_parallel_prange(js): #js being the random coefficients

    # Compute Hamiltonian
    H = np.zeros((N_DIM, N_DIM), dtype=np.complex128)
    for i in prange(N):
        for j in prange(i+1, N):
            for k in prange(j+1, N):
                for l in prange(k+1, N):
                    H += (1j**(Q/2))*js[i, j, k, l]*psi_pairs[i,j].dot(psi_pairs[k,l])

    return H

## 4. Run tests

In [10]:
sigma_j = np.sqrt((J**2)*np.math.factorial(Q-1)/(N**(Q-1)))
js_all = [np.random.normal(0, sigma_j, size=tuple([N for i in range(Q)])) for j in range(N_SAMPLES+1)]

# Instantiate functions so that they're compiled by cuda
_ = H4_base(js_all[0])
_ = H4_njit(js_all[0])
_ = H4_njit_parallel(js_all[0])
_ = H4_njit_parallel_prange(js_all[0])

def test_H4(H4_func, js_all):
    assert(len(js_all)==N_SAMPLES+1)

    # Execute on N_SAMPLES sequentially
    tic = time.time()
    for i in range(1, N_SAMPLES+1):
        H4_matrix = H4_func(js_all[i])
        #if i==1:
            #duration = time.time() - tic
            #exp_dur = duration*N_SAMPLES
            #print(f"  Expected duration: {exp_dur//60} minutes, {exp_dur%60} seconds")
    duration = time.time() - tic

    return duration

functions_to_test = {"H4_base": {"function": H4_base}, 
                     "H4_njit": {"function": H4_njit}, 
                     "H4_njit_parallel": {"function": H4_njit_parallel}, 
                     "H4_njit_parallel_prange": {"function": H4_njit_parallel_prange}}
tic = time.time()
for fname, fdict in functions_to_test.items():
    print(f"\n{fname}:")
    func = fdict["function"]
    duration = test_H4(func, js_all)
    fdict["duration"] = duration
    print(f"  Total duration: {duration//60} minutes, {duration%60} seconds")
    print(f"  Average time per Hamiltonian: {duration/N_SAMPLES//60} minutes, {duration/N_SAMPLES%60} seconds")

duration = time.time() - tic
print(f"\n\nTesting all functions: {duration//60} minutes, {duration%60} seconds")


H4_base:
  Actual duration: 0.0 minutes, 42.91655993461609 seconds
  Average time per Hamiltonian: 0.0 minutes, 0.4291655993461609 seconds

H4_njit:
  Actual duration: 8.0 minutes, 39.700655698776245 seconds
  Average time per Hamiltonian: 0.0 minutes, 5.197006556987763 seconds

H4_njit_parallel:
